In [ ]:
MSA ARCHITECTURE

sv1 :drivers
    driver_id
    name
    license
    status (assigned, busy, not_busy)

sv2:vehicles
    vehicle_id
    name
    plate number
    status(assigned, busy, not_busy)

sv3:assignment
    driver_id
    vehicle_id
    start datetime
    end datetime
    status (scheduled, completed, cancelled)

sv4:maintenance
    vehicle_id
    description
    start_datetime
    end_datetime
    status (scheduled, completed, cancelled)

sv5 schedule
    schedule_id
    schedule_type_name (assignment, task, maintenance)
    schedule_type_id(assignment_id, task_id, maintenance_id)
    start ( inherited from type )
    end datetime( inherited from type )
    status (scheduled, completed, cancelled)

sv6 fuel
    fuel_id
    vehicle_id
    datetime
    amount
    cost
    mileage
    fuel type

sv7 tasks
    task_id
    assignment (pair to do the task)
    task description (task name )
    start datetime
    end datetime
    priority
    status (scheduled, completed, cancelled)


from my understanding - levels of nesting/ dependencies
this shows for every service, which service's data does it hold in forms of ids

depedency_map = {

    sv1:[null],

    sv2:[null],

    sv3:[
        {sv1:[null]},
        {sv2:[null]}
    ],

    sv4:[{sv2:[null]}],

    sv5:[
        {sv3:[{sv1:[null]}, {sv2:[null]}]},
        {sv4:[{sv2:[null]}]},
        {sv6:[{sv2:[null]}]}
    ],

    sv6:[{sv2:[null]}],

    sv7:[{sv3:["sv1", "sv2"]}],

};

In [ ]:
IMPLEMENTATION for MQ

--------------- TEMPLATE ----------------

SERVICE NAME
    EVENT GIST 
        MESSAGE GIST SUMMARY
            EXCHANGE
                CONSUMER ( QUEUE NAME )
                    ACTION(HANDLER)
       


sv1 
    
    new driver created
    
        driver created fanout exchange
    
            sv1 ( service1_driver_created_queue )
                log message ( was just curious -- this is a pointless in house call )
    
            sv2 ( service2_driver_created_queue )
                log message ( was just curious )
    
            sv3 ( service3_driver_created_queue )
                log message ( was just curious )
sv2
    
    new vehicle created
    
        vehicle created fanout exchange
    
            sv1 ( service1_vehicle_created_queue )
                log message ( was just curious )
    
            sv2 ( service2_vehicle_created_queue )
                log message ( was just curious -- this is a pointless in house call )
    
            sv3 ( service3_vehicle_created_queue )
                log message ( was just curious )

sv3
    new assignment created
    
        assignment created fanout exchange
    
            sv1 ( service1_assignment_created_queue )
                get the driver status and change it to assigned

                QUERY: if the assignment is immmediate effect, thats easy
                but what if the assignment is in the future say 2 days from when user assignes pair, 
                also in normal world, we can plan assignments months ahead
                how do we handle that scenario such that when the date reached, 
                the DB change is made then
    
            sv2 ( service2_driver_created_queue )
                get the vehicle status and change it to assigned
                
                QUERY: if the assignment is immmediate effect, thats easy
                but what if the assignment is in the future say 2 days from when user assignes pair, 
                also in normal world, we can plan assignments months ahead
                how do we handle that scenario such that when the date reached, 
                the DB change is made then
    
            sv3 ( service3_driver_created_queue )
                log message ( was just curious -- this is a pointless in house call )
    
            sv5 (service5_assignment_created_queue)
                add the assignment id, dates and description to its database

sv4
    new maintenance record created

        maintenance created fanout exchange

            sv2 ( service2_driver_created_queue )
                get the vehicle status and change it to assigned

                QUERY: if the assignment is immmediate effect, thats easy
                but what if the assignment is in the future say 2 days from when user assignes pair, 
                also in normal world, we can plan assignments months ahead
                how do we handle that scenario such that when the date reached, 
                the DB change is made then
            
            sv5 ( service5_maintenance_created_queue )
                add the entire record to its db but only reference its id and dates

                QUERY:  is there more efficient way because this keeps records from 
                task maintenance assignment and we want only references to avoid
                redundancy, but retrievla becomes an issuewhen building a timeline schedule

            

        

            



In [ ]:
sv3
    new assignment created (future)

        assignment_created fanout exchange

            sv1 ( service1_assignment_created_queue )
                Schedule a task for the start date to change driver status to "assigned."

            sv2 ( service2_assignment_created_queue )
                Schedule a task for the start date to change vehicle status to "assigned."
            
            sv5 ( service5_assignment_created_queue )
                Add assignment details with "scheduled" status to the schedule service.
                Trigger an event at the start time for further processing.

sv4
    new maintenance created (future)

        maintenance_created fanout exchange

            sv2 ( service2_maintenance_created_queue )
                Schedule a task for the start date to change vehicle status to "under maintenance."

            sv5 ( service5_maintenance_created_queue )
                Add maintenance details with "scheduled" status to the schedule service.
                Trigger an event at the start time for further processing.
